In [ ]:
"""Simulation of a pure humeral rotation, arm on the side."""

# Imports
import scipy.spatial.transform as transform
import numpy as np
import matplotlib.pyplot as plt


def yxy_to_tt(yxy_angles: np.ndarray) -> np.ndarray:
    """Convert Euler YXY angles to Tilt-Torsion angles."""
    tt_angles = yxy_angles.copy()
    tt_angles[:, 2] += tt_angles[:, 0]
    tt_angles = np.mod(tt_angles + 180, 360) - 180
    return tt_angles


def tt_to_yxy(tt_angles: np.ndarray) -> np.ndarray:
    """Convert Tilt-Torsion angles to Euler YXY angles."""
    yxy_angles = tt_angles.copy()
    yxy_angles[:, 2] -= yxy_angles[:, 0]
    yxy_angles = np.mod(yxy_angles + 180, 360) - 180
    return yxy_angles


#-------------
# Script start

# Initialize random generator
np.random.seed(0)

# Define a movement of pure humeral rotation, from 100 degrees of external
# rotation to 100 degrees or internal rotation.
reference_angles = np.empty((200, 3))
reference_angles[:, 0] = 0  # Plane of elevation
reference_angles[:, 1] = 0  # Elevation
reference_angles[:, 2] = np.arange(-100, 100)  # Rotation

# Add white noise of ±1 amplitude
reference_angles += 2 * (np.random.rand(200, 3) - 0.5)

# Convert the reference angles to a series of rotation matrices
matrix = transform.Rotation.from_euler(
    'YXY', tt_to_yxy(reference_angles), degrees=True)

# Extract the angles back using Euler YXY and Tilt-Torsion methods
yxy_extracted_angles = matrix.as_euler('YXY', degrees=True)
tt_extracted_angles = yxy_to_tt(yxy_extracted_angles)

# Plot the results
plt.subplot(1, 3, 1)
plt.plot(reference_angles)
plt.title('Reference angles')
plt.axis([0, 200, -200, 200])
plt.grid(True)
plt.legend(['Plane of elevation', 'Elevation', 'Internal rotation'])

plt.subplot(1, 3, 2)
plt.plot(yxy_extracted_angles)
plt.title('Euler YXY')
plt.axis([0, 200, -200, 200])
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(tt_extracted_angles)
plt.title('Tilt-Torsion')
plt.axis([0, 200, -200, 200])
plt.grid(True)

# Compare the internal rotation between reference and tt_extracted
assert np.allclose(tt_extracted_angles[:, 2], reference_angles[:, 2])
